<a href="https://colab.research.google.com/github/vin-thomas/Enron_email_classification/blob/main/Enron__Deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import torch
import glob
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AIML Labs/@Project/working files 2b/Pre_processed/enron_csv_1', index_col=[0])

##**Extracting Emails**

In [ ]:
import re

add_fm = []
add_to = []
add_cc = []
add_bcc = []
for item in df.Raw_Text:
  fm_emails = re.findall(r"From: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  to_emails = re.findall(r"To: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  cc_emails = re.findall(r"Cc: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  bcc_emails = re.findall(r"Bcc: ([a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+)", str(item))
  if fm_emails:
    add_fm.append(fm_emails[0])
  else:
    add_fm.append(1000)
  if to_emails:
    add_to.append(to_emails[0])
  else:
    add_to.append(1000)
  if cc_emails:
    add_cc.append(cc_emails[0])
  else:
    add_cc.append(1000)
  if bcc_emails:
    add_bcc.append(bcc_emails[0])
  else:
    add_bcc.append(1000)  

In [ ]:
df['From']= add_fm
df['To']= add_to
df['CC']= add_cc
df['BCC']= add_bcc

##**Extracting Subject**

In [ ]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\\', '', text)
    text = re.sub(r'\\n', '', text)
    text = re.sub(r'-', '', text)
    return text

In [ ]:
subject_email = []

for item in df.Raw_Text:
  subject = re.findall(r"Subject: ([^\n]*|SUBJECT: [^\n]*)", str(item))
  subject = preprocess_text(subject[0])
  subject_email.append(subject)


In [ ]:
df['Subject']= subject_email

##**Extracting Body**

In [ ]:
body_email =[]
for item in df.Raw_Text:
  body = preprocess_text(re.findall (r"FileName:([\s\S]*)$", str(item))[0])
  word_list = body.split(' ')
  if len(word_list) > 130:
    word_list = word_list[2:130]
  word_list = " ".join(word_list)
  body_email.append(word_list)


In [ ]:
df['Body']= body_email

In [ ]:
df_final= df.drop_duplicates(subset=['Body', 'From'])

In [ ]:
df_final = df_final[df_final.Folder != 'all_documents']
df_final = df_final[df_final.Folder != 'deleted_items']
df_final = df_final[df_final.Body != 'na']

In [ ]:
df_final['Text']= df_final['Subject']+df_final['Body']
df_final.drop(['Subject', 'Body'], axis=1, inplace= True)

In [ ]:
for col in ['Name', 'Folder', 'From', 'To', 'CC', 'BCC']:
    df_final[col] = df_final[col].astype('category')

In [ ]:
df_final['from_id']= df_final['From'].factorize()[0]
df_final['to_id']= df_final['To'].factorize()[0]
df_final['cc_id']= df_final['CC'].factorize()[0]
df_final['bcc_id']= df_final['BCC'].factorize()[0]
df_final['folder']= df_final['Folder'].factorize()[0]
df_final['name']= df_final['Name'].factorize()[0]

**Create and save dictionaries**

In [ ]:
df_final

,Raw_Text,Name,Folder,From,To,CC,BCC,Text,from_id,to_id,cc_id,bcc_id,folder,name
0,b'Message-ID: <32069495.1075859544335.JavaMail...,sager-e,notes_inbox,enron.announcements@enron.com,all.states@enron.com,1000,1000,improved process for engaging temporary worker...,0,0,0,0,0,0
1,b'Message-ID: <3595780.1075853426259.JavaMail....,sager-e,notes_inbox,veronica.gonzalez@enron.com,elizabeth.sager@enron.com,1000,1000,margin letter socalrnmimeversion: .rncontentt...,1,1,0,0,0,0
2,b'Message-ID: <17371673.1075859599888.JavaMail...,sager-e,notes_inbox,arsystem@mailman.enron.com,elizabeth.sager@enron.com,1000,1000,your approval is overdue: access request for s...,2,1,0,0,0,0
3,b'Message-ID: <31570348.1075859599931.JavaMail...,sager-e,notes_inbox,arsystem@mailman.enron.com,elizabeth.sager@enron.com,1000,1000,your approval is overdue: access request forrn...,2,1,0,0,0,0
4,b'Message-ID: <8588396.1075853433656.JavaMail....,sager-e,notes_inbox,lech.kalembka@cwt.com,elizabeth.sager@enron.com,david.mitchell@cwt.com,david.mitchell@cwt.com,setoff provisionsrncc: david.mitchell@cwt.comr...,3,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516002,"b""Message-ID: <11587692.1075860394945.JavaMail...",hain-m,discussion_threads,lysa.akin@enron.com,1000,1000,1000,ron carrol call pin #: (full details below)rn...,3855,114,0,0,11,224
516003,b'Message-ID: <33529905.1075860385648.JavaMail...,hain-m,discussion_threads,lysa.akin@enron.com,1000,1000,1000,technical conference for caiso monitoring (det...,3855,114,0,0,11,224
516013,b'Message-ID: <32725392.1075860407508.JavaMail...,hain-m,discussion_threads,sabine.schnittger@frontier-economics.com,mary.hain@enron.com,1000,1000,re: i'm leaving enronrnmimeversion: .rncontent...,8324,134,0,0,11,224
516036,b'Message-ID: <15311345.1075860408093.JavaMail...,hain-m,discussion_threads,meames@idahopower.com,mary.hain@enron.com,1000,1000,re: i'm leaving enronrnmimeversion: .rncontent...,15275,134,0,0,11,224


In [ ]:
from_dict= dict( zip( df_final['from_id'], df_final['From'] ) )
to_dict= dict( zip( df_final['to_id'], df_final['To'] ) )
cc_dict= dict( zip( df_final['cc_id'], df_final['CC'] ) )
bcc_dict= dict( zip( df_final['bcc_id'], df_final['BCC'] ) )
name_dict= dict( zip( df_final['name'], df_final['Name'] ) )
folder_dict = dict( zip( df_final['folder'], df_final['Folder'] ) )

In [ ]:
pickle.dump(from_dict, open('/content/from_dict', 'wb'))
pickle.dump(to_dict, open('/content/to_dict', 'wb' ))
pickle.dump(cc_dict, open('/content/cc_dict', 'wb' ))
pickle.dump(bcc_dict, open('/content/bcc_dict', 'wb' ))
pickle.dump(name_dict, open('/content/name_dict', 'wb' ))
pickle.dump (folder_dict, open('/content/folder_dict', 'wb'))

In [ ]:
df_final.drop(['From', 'To', 'CC', 'BCC'], axis=1, inplace= True)

In [ ]:
df_final

,Raw_Text,Name,Folder,Text,from_id,to_id,cc_id,bcc_id,folder,name
0,b'Message-ID: <32069495.1075859544335.JavaMail...,sager-e,notes_inbox,improved process for engaging temporary worker...,0,0,0,0,0,0
1,b'Message-ID: <3595780.1075853426259.JavaMail....,sager-e,notes_inbox,margin letter socalrnmimeversion: .rncontentt...,1,1,0,0,0,0
2,b'Message-ID: <17371673.1075859599888.JavaMail...,sager-e,notes_inbox,your approval is overdue: access request for s...,2,1,0,0,0,0
3,b'Message-ID: <31570348.1075859599931.JavaMail...,sager-e,notes_inbox,your approval is overdue: access request forrn...,2,1,0,0,0,0
4,b'Message-ID: <8588396.1075853433656.JavaMail....,sager-e,notes_inbox,setoff provisionsrncc: david.mitchell@cwt.comr...,3,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...
516002,"b""Message-ID: <11587692.1075860394945.JavaMail...",hain-m,discussion_threads,ron carrol call pin #: (full details below)rn...,3855,114,0,0,11,224
516003,b'Message-ID: <33529905.1075860385648.JavaMail...,hain-m,discussion_threads,technical conference for caiso monitoring (det...,3855,114,0,0,11,224
516013,b'Message-ID: <32725392.1075860407508.JavaMail...,hain-m,discussion_threads,re: i'm leaving enronrnmimeversion: .rncontent...,8324,134,0,0,11,224
516036,b'Message-ID: <15311345.1075860408093.JavaMail...,hain-m,discussion_threads,re: i'm leaving enronrnmimeversion: .rncontent...,15275,134,0,0,11,224


In [ ]:
#df_tfidf= df_final.sample(60000)
df_RawText = df_final[['Raw_Text', 'Folder', 'Name']]
df_RawText= df_RawText.sample(500)
df_RawText.to_csv('/content/drive/MyDrive/AIML Labs/@Project/working files 2b/Deployent2/raw.csv')

In [ ]:
df_tfidf = df_final[["Text", "from_id", "to_id", "cc_id", "bcc_id", "name", "folder"]]

In [ ]:
df_tfidf= df_tfidf.reset_index(drop=True)

In [ ]:
df_tfidf

,Text,from_id,to_id,cc_id,bcc_id,name,folder
0,improved process for engaging temporary worker...,0,0,0,0,0,0
1,margin letter socalrnmimeversion: .rncontentt...,1,1,0,0,0,0
2,your approval is overdue: access request for s...,2,1,0,0,0,0
3,your approval is overdue: access request forrn...,2,1,0,0,0,0
4,setoff provisionsrncc: david.mitchell@cwt.comr...,3,1,1,1,0,0
...,...,...,...,...,...,...,...
197598,ron carrol call pin #: (full details below)rn...,3855,114,0,0,224,11
197599,technical conference for caiso monitoring (det...,3855,114,0,0,224,11
197600,re: i'm leaving enronrnmimeversion: .rncontent...,8324,134,0,0,224,11
197601,re: i'm leaving enronrnmimeversion: .rncontent...,15275,134,0,0,224,11


##**Vectorization**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=10000, max_df=100000, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(df_tfidf['Text']).toarray()


In [ ]:
df_RawText

,Raw_Text,Folder,Name
344410,"b""Message-ID: <28398675.1075854926116.JavaMail...",inbox,heard-m
412333,b'Message-ID: <10379278.1075852419639.JavaMail...,sent_items,hernandez-j
124796,"b""Message-ID: <11528325.1075854800614.JavaMail...",_sent_mail,bass-e
52466,"b""Message-ID: <1257186.1075860036407.JavaMail....",notes_inbox,taylor-m
178586,b'Message-ID: <11789013.1075853079368.JavaMail...,sent_items,blair-l
...,...,...,...
310485,b'Message-ID: <6926382.1075858059340.JavaMail....,notes_inbox,whalley-l
260558,"b""Message-ID: <30633933.1075856546513.JavaMail...",_sent_mail,kaminski-v
37615,b'Message-ID: <7057546.1075841471986.JavaMail....,personal,richey-c
180382,"b""Message-ID: <19641628.1075858292773.JavaMail...",_sent_mail,love-p


In [ ]:
df_features = pd.DataFrame(features)

In [ ]:
df_tfidf

,Text,from_id,to_id,cc_id,bcc_id,name,folder
0,improved process for engaging temporary worker...,0,0,0,0,0,0
1,margin letter socalrnmimeversion: .rncontentt...,1,1,0,0,0,0
2,your approval is overdue: access request for s...,2,1,0,0,0,0
3,your approval is overdue: access request forrn...,2,1,0,0,0,0
4,setoff provisionsrncc: david.mitchell@cwt.comr...,3,1,1,1,0,0
...,...,...,...,...,...,...,...
197598,ron carrol call pin #: (full details below)rn...,3855,114,0,0,224,11
197599,technical conference for caiso monitoring (det...,3855,114,0,0,224,11
197600,re: i'm leaving enronrnmimeversion: .rncontent...,8324,134,0,0,224,11
197601,re: i'm leaving enronrnmimeversion: .rncontent...,15275,134,0,0,224,11


In [ ]:
df_tfidf.drop(['Text'], axis=1, inplace= True)

In [ ]:
df_tfidf= df_tfidf.join(df_features)

In [ ]:
y = df_tfidf.folder
X= df_tfidf.drop(['folder'], axis=1)

In [ ]:
X.shape

(197603, 260)

##**Standardization**

In [ ]:
from sklearn.preprocessing import StandardScaler


# perform a robust scaler transform of the dataset
ss = StandardScaler()
X = ss.fit_transform(X)
# convert the array back to a dataframe
X = pd.DataFrame(X)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


##**Train/Test Split**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=0)

##**Train**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [ ]:
model = LogisticRegression(random_state=0)
model.fit(X_train, y_train)
y_pred= model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print ('LR', accuracy)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LR 0.8554617652814046


In [ ]:
pickle.dump(ss, open('/content/ss', 'wb'))
filename = '/content/model'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
pickle.dump(tfidf, open('/content/tfidf', 'wb'))